In [0]:
# NACHISS - CLEAN 

In [17]:
!ls /content/drive/NLP_bootcamp/Data/training.1600000.processed.noemoticon.csv

/content/drive/NLP_bootcamp/Data/training.1600000.processed.noemoticon.csv


In [18]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# A simple sentiment prototype

In [0]:
import os  # manipulate paths
import pandas as pd  # SQL-like operations and convenience functions
import joblib  # save and load models

Download the Sentiment140 data from [their website](http://help.sentiment140.com/for-students) or directly from [Standford site](http://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip) and set `DATA_DIR` to the directory in which you have put the `CSV` files.

In [0]:
DATA_DIR = "/content/drive/NLP_bootcamp/Data/"
training_csv_file = os.path.join(DATA_DIR, 'training.1600000.processed.noemoticon.csv')

In [21]:
training_csv_file

'/content/drive/NLP_bootcamp/Data/training.1600000.processed.noemoticon.csv'

## A peek at the data

In [0]:
names = ('polarity', 'id', 'date', 'query', 'author', 'text')
df = pd.read_csv(training_csv_file, encoding='latin1', names=names)

In [23]:
pd.options.display.max_colwidth = 140  # allow wide columns
df.head()  # show first 5 rows

,polarity,id,date,query,author,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by texting it... and might cry as a result School today also. Blah!
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Managed to save 50% The rest go out of bounds
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all. i'm mad. why am i here? because I can't see you all over there."


In [24]:
df.tail()

,polarity,id,date,query,author,text
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best feeling ever
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interviews! â« http://blip.fm/~8bmta
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me for details
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! Tupac Amaru Shakur
1599999,4,2193602129,Tue Jun 16 08:40:50 PDT 2009,NO_QUERY,RyanTrevMorris,happy #charitytuesday @theNSPCC @SparksCharity @SpeakingUpH4H


In [0]:
df['polarity'].replace({0: -1, 4: 1}, inplace=True)
text = df['text']
target = df['polarity'].values

In [26]:
print(len(target), len(text))

1600000 1600000


## Train the model

Set 20% of the data aside to test the trained model

In [0]:
import sklearn
#from sklearn.cross_validation import train_test_split
from sklearn.model_selection import train_test_split
text_train, text_validation, target_train, target_validation = (
    train_test_split(text, target, test_size=0.2, random_state=42)
)

Build a pipeline

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import LogisticRegressionCV
from sklearn.pipeline import Pipeline

vectorizer = CountVectorizer(ngram_range=(1, 2), max_features=100000)
feature_selector = SelectKBest(chi2, k=5000)
classifier = LogisticRegressionCV(n_jobs=4)

This next cell took ~3 minutes to run on my machine

In [30]:
if os.path.exists('model.pkl'):
    sentiment_pipeline = joblib.load('model.pkl')
else:
    sentiment_pipeline = Pipeline((
        ('v', vectorizer),
        ('f', feature_selector),
        ('c', classifier)
    ))
    sentiment_pipeline.fit(text_train, target_train)
    joblib.dump(sentiment_pipeline, 'model.pkl');

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


## Test the model

In [31]:
print(sentiment_pipeline.predict(['bad', 'good', "didnt like", "today was a good day", "i hate this product"]))

[-1  1 -1  1 -1]


In [32]:
for text, target in zip(text_validation[:10], target_validation[:10]):
    print(sentiment_pipeline.predict([text])[0], target, '\t', text)

1 -1 	 @chrishasboobs AHHH I HOPE YOUR OK!!! 
1 -1 	 @misstoriblack cool , i have no tweet apps  for my razr 2
1 -1 	 @TiannaChaos i know  just family drama. its lame.hey next time u hang out with kim n u guys like have a sleepover or whatever, ill call u
-1 -1 	 School email won't open  and I have geography stuff on there to revise! *Stupid School* :'(
1 -1 	 upper airways problem 
-1 -1 	 Going to miss Pastor's sermon on Faith... 
1 1 	 on lunch....dj should come eat with me 
-1 -1 	 @piginthepoke oh why are you feeling like that? 
-1 -1 	 gahh noo!peyton needs to live!this is horrible 
1 1 	 @mrstessyman thank you glad you like it! There is a product review bit on the site  Enjoy knitting it!


In [33]:
sentiment_pipeline.score(text_validation, target_validation)

0.799475

## What did the model learn?

In [0]:
feature_names = sentiment_pipeline.steps[0][1].get_feature_names()
feature_names = [feature_names[i] for i in 
                 sentiment_pipeline.steps[1][1].get_support(indices=True)]

def show_most_informative_features(feature_names, clf, n=1000):
    coefs_with_fns = sorted(zip(clf.coef_[0], feature_names))
    top = zip(coefs_with_fns[:n], coefs_with_fns[:-(n + 1):-1])
    for (coef_1, fn_1), (coef_2, fn_2) in top:
        print("\t%.4f\t%-15s\t\t%.4f\t%-15s" % (coef_1, fn_1, coef_2, fn_2))

In [35]:
show_most_informative_features(feature_names, sentiment_pipeline.steps[2][1], n=500)

	-3.4419	not happy      		2.6677	no problem     
	-3.2841	clean me       		2.6432	no worries     
	-2.9337	inaperfectworld		2.5395	cannot wait    
	-2.9111	not looking    		2.3148	cant wait      
	-2.9024	sad            		2.2814	no prob        
	-2.8635	passed away    		2.2347	smiling        
	-2.6954	sadly          		2.1875	nothing wrong  
	-2.6288	gutted         		2.1168	not bad        
	-2.6195	not nice       		2.0543	sad sad        
	-2.6100	not cool       		1.9154	congratulations
	-2.5822	no luck        		1.8035	fuzzball       
	-2.5809	disappointing  		1.7795	no probs       
	-2.5067	heartbreaking  		1.7703	welcome        
	-2.5044	sadd           		1.7268	musicmonday    
	-2.4769	heartbroken    		1.7012	hate hate      
	-2.4462	boohoo         		1.6602	smile          
	-2.4255	dontyouhate    		1.6524	yayyy          
	-2.4240	rip            		1.6217	woooo          
	-2.4006	poor           		1.6184	thankyou       
	-2.3978	not fun        		1.5969	just sayin     
	-2.3921	bummer     